## FLAML for hp optimisation and model selection
We use FLAML twice, first to find the best component model for each estimator, and then to optimise the estimators themselves and choose the best estimator. Here we show how it's done

In [31]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now.. 

root_path = root_path = os.path.realpath('../..')
sys.path.append(os.path.join(root_path, "auto-causality"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of auto_causality.optimiser failed: Traceback (most recent call last):
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", l

In [32]:
from auto_causality import AutoCausality
from auto_causality.datasets import synth_ihdp, preprocess_dataset

### Model fitting & scoring
Here we fit a (selection of) model(s) to the data and score them with the ERUPT metric on held-out data

In [34]:
# import dataset
data_df = synth_ihdp()
data_df, features_X, features_W, targets, treatment = preprocess_dataset(data_df)

# choose which estimators to fit
estimator_list = ["LinearDML","metalearners"]

# init autocausality object with chosen metric to optimise
ac = AutoCausality(
    time_budget=1000, estimator_list=estimator_list, metric="erupt", verbose=3,components_verbose=1,components_time_budget=2,use_ray=False
)

# run autocausality
myresults = ac.fit(data_df, treatment, targets[0], features_W, features_X)

# return best estimator
print(f"Best estimator: {ac.best_estimator}")
# config of best estimator:
print(f"best config: {ac.best_config}")
# best score:
print(f"best score: {ac.best_score}")


[autoreload of auto_causality.optimiser failed: Traceback (most recent call last):
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\IPython\extensions\autoreload.py", l

config: {'fit_cate_intercept': 0, 'mc_iters': 7}
config: {'n_estimators': 4, 'max_depth': 1, 'min_child_weight': 41.56762955113965, 'learning_rate': 0.2820608761915944, 'subsample': 0.7813433032287629, 'colsample_bylevel': 0.06504946646365602, 'colsample_bytree': 0.016809182597452822, 'reg_alpha': 0.018567785142079223, 'reg_lambda': 9.649618313107304}


TypeError: __init__() got an unexpected keyword argument 'n_estimators'